# Cell Migration Simulation - WorkflowThis notebook demonstrates the complete workflow for simulating cell migration in a stadium-shaped domain with chemotactic gradient.

## 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from simulation import Simulation
from scipy.stats import lognorm
from analysis import (calculate_autocorrelation, calculate_msd,
                      fit_von_mises_mixture_mle, compute_turning_angles,
                      compute_msd_dacf_per_movie)
from visualization import plot_trajectories, create_animation, plot_cell_statistics

## 2. Load and Analyze Experimental DataLoad experimental trajectory data and fit velocity and turning angle distributions.

In [ ]:
# Load experimental data
data = pd.read_csv('path/to/your/data.csv')

# Calculate velocities if not already present
data['v_x'] = data.groupby('track_id')['x_microns'].diff()
data['v_y'] = data.groupby('track_id')['y_microns'].diff()
data['speed'] = np.sqrt(data['v_x']**2 + data['v_y']**2)

In [ ]:
# Fit log-normal distribution to velocities
velocities = data['speed'].dropna()
velocities = velocities[velocities > 0]
shape, loc, scale = lognorm.fit(velocities, floc=0)
velocity_params = {'shape': shape, 'loc': loc, 'scale': scale}

print(f"Velocity distribution parameters: shape={shape:.3f}, loc={loc:.3f}, scale={scale:.3f}")

In [ ]:
# Compute and fit turning angles distribution
turning_angles = compute_turning_angles(data, track_col='track_id',
                                       x_col='x_microns', y_col='y_microns',
                                       step_col='step', lag=1)

# Fit von Mises mixture model
vonmises_fit = fit_von_mises_mixture_mle(turning_angles, initial_guess=(0.5, 1.0, 5.0))
vonmises_params = vonmises_fit['params']

print(f"Von Mises parameters: W1={vonmises_params['W1']:.3f}, "
      f"kappa1={vonmises_params['kappa1']:.3f}, kappa2={vonmises_params['kappa2']:.3f}")

In [ ]:
# Compute experimental MSD and DACF
msd_exp, dacf_exp = compute_msd_dacf_per_movie(data, x_col='x_microns',
                                                y_col='y_microns', max_lag=None)

## 3. Run SimulationCreate and run a simulation with fitted parameters.

In [ ]:
# Create simulation with fitted parameters
sim = Simulation(
    n_cells=50,
    time_step=5.0,              # minutes
    stadium_L=800,              # microns
    stadium_R=200,              # microns
    source_length=400,          # microns
    chemotaxis_strength=0.3,    # adjust as needed
    repulsion_strength=0.2,     # adjust as needed
    interaction_radius=10.0,
    velocity_params=velocity_params,
    persistence=0.8,            # adjust as needed
    starting_positions='perimeter',
    mode='exp_memory',          # or 'persistent', 'biased_persistent'
    memory_window=10,
    memory_exp_lambda=0.1,
    vonmises_params=vonmises_params
)

# Run simulation
sim.run(n_steps=100, verbose=True)

## 4. Visualize Results

In [ ]:
# Plot trajectories
plot_trajectories(sim, show_gradient=True)

# Plot statistics
plot_cell_statistics(sim)

# Create animation (optional)
# create_animation(sim, interval=100, save_path='migration.gif')

## 5. Compare with Experimental Data

In [ ]:
# Get simulation data
sim_df = sim.get_dataframe()

# Calculate simulation metrics
sim_dacf = calculate_autocorrelation(sim_df, max_lag=None, directional=True)
sim_msd = calculate_msd(sim_df, max_lag=None)

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# DACF comparison
ax1.plot(dacf_exp['lag'], dacf_exp['dacf_mean'], 'o-', label='Experimental', alpha=0.7)
ax1.fill_between(dacf_exp['lag'],
                  dacf_exp['dacf_mean'] - dacf_exp['dacf_sem'],
                  dacf_exp['dacf_mean'] + dacf_exp['dacf_sem'], alpha=0.3)
ax1.plot(sim_dacf['lag'], sim_dacf['dacf'], 's-', label='Simulation', alpha=0.7)
ax1.set_xlabel('Time Lag (steps)')
ax1.set_ylabel('DACF')
ax1.set_title('Directional Autocorrelation Function')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MSD comparison
ax2.plot(msd_exp['lag'], msd_exp['msd_mean'], 'o-', label='Experimental', alpha=0.7)
ax2.fill_between(msd_exp['lag'],
                  msd_exp['msd_mean'] - msd_exp['msd_sem'],
                  msd_exp['msd_mean'] + msd_exp['msd_sem'], alpha=0.3)
ax2.plot(sim_msd['lag'], sim_msd['msd'], 's-', label='Simulation', alpha=0.7)
ax2.set_xlabel('Time Lag (steps)')
ax2.set_ylabel('MSD (μm²)')
ax2.set_title('Mean Squared Displacement')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Parameter Fitting (Optional)Perform grid search to find best-fit parameters.

In [ ]:
# Uncomment to run parameter fitting
# from fit import fit_grid

# param_grid = {
#     'persistence': [0.0, 0.3, 0.5, 0.7, 0.9],
#     'chemotaxis_strength': [0.0, 0.1, 0.3, 0.5],
#     'memory_exp_lambda': [0.05, 0.1, 0.2]
# }

# best_params, results_table = fit_grid(
#     exp_df=data,
#     simulation_mode='exp_memory',
#     param_grid=param_grid,
#     chi_weight=0.5,
#     velocity_dist_params=velocity_params,
#     vonmises_params=vonmises_params,
#     n_steps=100,
#     n_cells=50,
#     seed=42
# )

# print("Best parameters:")
# print(best_params)
# results_table.to_csv('fitting_results.csv', index=False)

## 7. Save Results

In [ ]:
# Save simulation trajectories
sim.save_trajectories('simulation_trajectories.csv')

# Save analysis results
sim_dacf.to_csv('simulation_dacf.csv', index=False)
sim_msd.to_csv('simulation_msd.csv', index=False)

print("Results saved successfully!")